In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas
import matplotlib
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("AdityaShirke_A")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

In [3]:
sc = spark_session.sparkContext
lines_sv_en_en = sc.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.en")
linecount_sv_en_en = lines_sv_en_en.count()
linecount_sv_en_en

1862234

In [4]:
lines_sv_en_sv = sc.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.sv").cache()
linecount_nl_en_sv = lines_sv_en_sv.count()
linecount_nl_en_sv

1862234

In [5]:
lines_sv_en_en.getNumPartitions()

2

In [6]:
lines_sv_en_sv.getNumPartitions()

3

In [7]:
lines_sv_en_en.take(10)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
 'You have requested a debate on this subject in the course of the next few days, during this part-session.',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.",
 "Please rise, then, for this minute' s silence.",
 "(The House rose and observed a minute' s silence)",
 'Madam President, on a point of order.',
 'You will be aware from the press and television that there have be

In [8]:
#A2.1 function to Pre-process the text from both RDDs by doing the following: Lowercase the text and Tokenize the text (split on space)
def lowercase_split(rdd):
    return rdd.map(lambda line: line.lower().split(' '))

preprocessed_text_sv_en_en = lowercase_split(lines_sv_en_en)
print(preprocessed_text_sv_en_en.take(10))
print(preprocessed_text_sv_en_en.count())

preprocessed_text_sv_en_sv = lowercase_split(lines_sv_en_sv)
print(preprocessed_text_sv_en_sv.take(10))
print(preprocessed_text_sv_en_sv.count())

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

In [9]:
import string #to remove punctuations
#reference: https://medium.com/@gulcanogundur/pyspark-word-count-b099106135a7 for punctuations and lambda use.

def lowercase_split_map(rdd):
    split_lines = rdd.flatMap(lambda line: line.lower().strip(string.punctuation).split(' '))
    mapped = split_lines.filter(lambda x: x.isalpha()) \
                .map(lambda words: (words.strip(string.punctuation), 1), split_lines) \
                .reduceByKey(lambda a, b: a + b) \
                .sortByKey()
    return mapped

word_count_sv_en_en = lowercase_split_map(lines_sv_en_en)
word_count_sv_en_en = word_count_sv_en_en.map(lambda x:(x[1],x[0]))

#print(word_count_sv_en_en.sortByKey(False).take(100))


word_count_sv_en_sv = lowercase_split_map(lines_sv_en_sv)
word_count_sv_en_sv = word_count_sv_en_sv.map(lambda x:(x[1],x[0]))

print(word_count_sv_en_sv.sortByKey(False).take(100))


[(1706328, 'att'), (1344865, 'och'), (1052440, 'i'), (942048, 'det'), (913312, 'som'), (912272, 'för'), (739577, 'av'), (696798, 'är'), (620508, 'en'), (540272, 'vi'), (514492, 'om'), (513971, 'de'), (510056, 'till'), (502907, 'jag'), (500877, 'den'), (482266, 'på'), (469207, 'har'), (426964, 'med'), (402917, 'inte'), (370117, 'ett'), (252522, 'detta'), (194851, 'kommer'), (193936, 'kan'), (181528, 'måste'), (148730, 'från'), (144093, 'europeiska'), (130340, 'vill'), (127621, 'så'), (125748, 'men'), (125226, 'också'), (124630, 'skulle'), (123821, 'när'), (123456, 'denna'), (119501, 'man'), (117352, 'herr'), (110630, 'kommissionen'), (110466, 'mycket'), (110462, 'alla'), (100930, 'sig'), (100210, 'dessa'), (93384, 'vara'), (93327, 'eller'), (89829, 'även'), (89100, 'andra'), (88969, 'gäller'), (80671, 'oss'), (80333, 'mot'), (80226, 'under'), (79907, 'därför'), (79281, 'här'), (79052, 'inom'), (77565, 'eu'), (75126, 'finns'), (74734, 'skall'), (73615, 'utan'), (69981, 'genom'), (67909, 

In [10]:
en_1 = lines_sv_en_en.zipWithIndex()
en_2 = en_1.map(lambda x:(x[1],x[0]))

sv_1 = lines_sv_en_sv.zipWithIndex()
sv_2 = sv_1.map(lambda x:(x[1],x[0]))

joined_en2_sv2 = en_2.join(sv_2)

#print(sv_2.take(10))

print(joined_en2_sv2.take(10))



[(0, ('Resumption of the session', 'Återupptagande av sessionen')), (5, ("Please rise, then, for this minute' s silence.", 'Jag ber er resa er för en tyst minut.')), (10, ("Would it be appropriate for you, Madam President, to write a letter to the Sri Lankan President expressing Parliament's regret at his and the other violent deaths in Sri Lanka and urging her to do everything she possibly can to seek a peaceful reconciliation to a very difficult situation?", 'Skulle det vara möjligt för er, fru talman, att skriva ett brev till den srilankesiska presidenten i vilket parlamentets beklagande uttrycks över hans och de övriga brutala dödsfallen i Sri Lanka och uppmanar henne att göra allt som står i hennes makt för att få en fredlig lösning på en mycket komplicerad situation?')), (15, ('My question relates to something that will come up on Thursday and which I will then raise again.', 'Min fråga har att göra med något som kommer att behandlas på torsdag och som jag då kommer att ta upp ig

In [11]:
#Filter to exclude line pairs that have an empty/missing “corresponding” sentence
joined_rdd = joined_en2_sv2 #count: 186223
joined_rdd = joined_rdd.filter(lambda x: (len(x[1][0].split()) != 0 )) #count:1850885
joined_rdd_f1 = joined_rdd.filter(lambda x: (len(x[1][1].split()) != 0 )) #count:1848423


In [12]:
#Filter to leave only pairs of sentences with a small number of words per sentence,
joined_rdd_f2 = joined_rdd_f1.filter(lambda x: (len(x[1][0].split()) == 7)) #count with word count = 10 :47029, #count with word count = 5 :23483 #count with word count = 7 :35146
print(joined_rdd_f2.count())

#Filter to leave only pairs of sentences with the same number of words in each sentence.
joined_rdd_f3 = joined_rdd_f2.filter(lambda x: (len(x[1][0].split()) == len(x[1][1].split()))) #count:9175, 8952, 9545 (selected)
print(joined_rdd_f3.count())

print(joined_rdd_f3.take(10))


35146
9545
[(1825, ('We await the outcome with great interest.', 'Vi väntar på resultaten med stort intresse.')), (2770, ('The Commission, like others, has urged dialogue.', 'Kommissionen har, liksom andra, uppmanat till dialog.')), (3345, ('This communication must contain a clear timetable.', 'Detta meddelande måste innehålla en klar tidtabell.')), (4005, ('The laws against racism must be harmonised.', 'Det krävs harmonisering av lagstiftningen mot rasism.')), (4640, ('It is also a matter of conviction.', 'Det är också en fråga om övertygelse.')), (5275, ('That is why he deserves special mention!', 'Därför vill jag här särskilt nämna det!')), (5760, ('This was an important consideration for us.', 'Detta var ett viktigt ställningstagande för oss.')), (6865, ('It has no right to do so.', 'Det tillkommer inte EU att göra så.')), (7755, ('The second subject is flexibility and simplification.', 'Den andra frågan gäller flexibilitet och förenkling.')), (9205, ('We have seen vote after vote 

In [13]:
zipped_pair = joined_rdd_f3.map(lambda x:(x[1][0],x[1][1]))
print(zipped_pair.take(10))

zipped_pair_2 = zipped_pair.map(lambda line: (line[0].split(), line[1].split()))
print(zipped_pair_2.take(10))





[('We await the outcome with great interest.', 'Vi väntar på resultaten med stort intresse.'), ('The Commission, like others, has urged dialogue.', 'Kommissionen har, liksom andra, uppmanat till dialog.'), ('This communication must contain a clear timetable.', 'Detta meddelande måste innehålla en klar tidtabell.'), ('The laws against racism must be harmonised.', 'Det krävs harmonisering av lagstiftningen mot rasism.'), ('It is also a matter of conviction.', 'Det är också en fråga om övertygelse.'), ('That is why he deserves special mention!', 'Därför vill jag här särskilt nämna det!'), ('This was an important consideration for us.', 'Detta var ett viktigt ställningstagande för oss.'), ('It has no right to do so.', 'Det tillkommer inte EU att göra så.'), ('The second subject is flexibility and simplification.', 'Den andra frågan gäller flexibilitet och förenkling.'), ('We have seen vote after vote lost.', 'Vi har sett omröstning efter omröstning förloras.')]
[(['We', 'await', 'the', 'ou

In [14]:
#Checking how data looks on separate basis for creating lambda query
zipped_pair_en = zipped_pair.map(lambda x:x[0].split())
zipped_pair_sv = zipped_pair.map(lambda x:x[1].split())
#print(zipped_pair_en.take(10))
#print(zipped_pair_sv.take(10))

#One way
#zipped_pair_combine_1 = zipped_pair.map(lambda x: [(x[i].split(), x[i+1].split(), 1) for i in range(0, len(x) - 1)])
#print(zipped_pair_combine_1.take(10))

#Another way which is given as hint in assignment - Going with this
zipped_pair_combine = zipped_pair_en.zip(zipped_pair_sv)

print(zipped_pair_combine.take(10))


zipped_pair_word_pair = zipped_pair_combine.flatMap(lambda x: [((x[0][i], x[1][i]), 1) for i in range(0, len(x[0]) - 1)])
print(zipped_pair_word_pair.take(10))



[(['We', 'await', 'the', 'outcome', 'with', 'great', 'interest.'], ['Vi', 'väntar', 'på', 'resultaten', 'med', 'stort', 'intresse.']), (['The', 'Commission,', 'like', 'others,', 'has', 'urged', 'dialogue.'], ['Kommissionen', 'har,', 'liksom', 'andra,', 'uppmanat', 'till', 'dialog.']), (['This', 'communication', 'must', 'contain', 'a', 'clear', 'timetable.'], ['Detta', 'meddelande', 'måste', 'innehålla', 'en', 'klar', 'tidtabell.']), (['The', 'laws', 'against', 'racism', 'must', 'be', 'harmonised.'], ['Det', 'krävs', 'harmonisering', 'av', 'lagstiftningen', 'mot', 'rasism.']), (['It', 'is', 'also', 'a', 'matter', 'of', 'conviction.'], ['Det', 'är', 'också', 'en', 'fråga', 'om', 'övertygelse.']), (['That', 'is', 'why', 'he', 'deserves', 'special', 'mention!'], ['Därför', 'vill', 'jag', 'här', 'särskilt', 'nämna', 'det!']), (['This', 'was', 'an', 'important', 'consideration', 'for', 'us.'], ['Detta', 'var', 'ett', 'viktigt', 'ställningstagande', 'för', 'oss.']), (['It', 'has', 'no', 'righ

In [15]:
zipped_pair_word_pair_reducer = zipped_pair_word_pair.reduceByKey(lambda a, b: a + b)
zipped_pair_word_pair_reducer_t = zipped_pair_word_pair_reducer.map(lambda x:(x[1],x[0]))
print(zipped_pair_word_pair_reducer_t.sortByKey(False).take(100))


[(1289, ('is', 'är')), (890, ('I', 'Jag')), (875, ('We', 'Vi')), (564, ('Question', 'Fråga')), (559, ('No', 'nr')), (545, ('and', 'och')), (517, ('by', 'från')), (513, ('a', 'en')), (469, ('not', 'inte')), (417, ('have', 'har')), (346, ('It', 'Det')), (339, ('a', 'ett')), (320, ('in', 'i')), (299, ('to', 'att')), (298, ('for', 'för')), (288, ('This', 'Detta')), (268, ('That', 'Det')), (267, ('must', 'måste')), (252, ('are', 'är')), (239, ('this', 'detta')), (233, ('we', 'vi')), (227, ('of', 'av')), (205, ('There', 'Det')), (180, ('has', 'har')), (178, ('very', 'mycket')), (165, ('that', 'att')), (162, ('be', 'vara')), (161, ('This', 'Det')), (159, ('on', 'på')), (158, ('will', 'att')), (155, ('can', 'kan')), (145, ('to', 'till')), (140, ('the', 'den')), (139, ('will', 'kommer')), (137, ('the', 'det')), (136, ('Let', 'Låt')), (133, ('The', 'Omröstningen')), (133, ('vote', 'kommer')), (129, ('place', 'rum')), (128, ('They', 'De')), (128, ('take', 'äga')), (126, ('with', 'med')), (124, ('